Extract mel-spectrogram from audio

In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, TimeDistributed, LSTM, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestClassifier



import os
import glob
import matplotlib.pyplot as plt


Find graphic specs

In [2]:
# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(640, 480))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return np.squeeze(features)

train

In [3]:
# Load pre-trained models
resnet_model = ResNet50(weights='imagenet', include_top=False)
vgg_model = VGG16(weights='imagenet', include_top=False)

In [4]:
# Load images and extract features
image_files_HC = glob.glob(os.path.join("plots", "HC", "*.png"))
image_files_PD = glob.glob(os.path.join("plots", "PD", "*.png"))
image_files = image_files_HC + image_files_PD
features_resnet = []
features_vgg = []
labels = []
for image_file in image_files:
    features_resnet.append(extract_features(image_file, resnet_model).flatten())
    features_vgg.append(extract_features(image_file, vgg_model).flatten())
    if 'HC' in image_file:
        labels.append(0)
    else:
        labels.append(1)

1/1 [==============================] - 0s 239ms/step


In [5]:
# Combine features from both models
features = np.concatenate((features_resnet, features_vgg), axis=1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 3, the array at index 0 has size 2048 and the array at index 1 has size 512